In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying elevator_action.bin from HC ROMS/BY ALPHABET (PAL)/

In [2]:
episodes = 1
import gym
from gym import wrappers
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

env = gym.make('FrozenLake-v0')
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()

for episode in range(1,episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
        clear_output(wait=True)
    print('Episode: {}\nScore: {}'.format(episode,score))
    
env.close()

In [3]:
#Creating Q-Table
actions = env.action_space.n
state = env.observation_space.n

q_table = np.zeros((state,actions))

In [4]:
#Parameters for Q-learning
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

rewards_all_episodes = []


In [5]:
#Q-Learning Algorithm
import random

for episode in range(num_episodes):
  state = env.reset()
  done = False
  rewards_current_episode = 0

  for step in range(max_steps_per_episode):
    exploration_thresold = random.uniform(0, 1)
    if exploration_thresold > exploration_rate:
      action = np.argmax(q_table[state,:])
    else:
      action = env.action_space.sample()
    new_state, reward, done, info = env.step(action) 
    #observation, reward, self._done, self._info = self._gym_env.step(action) 
    #Updated Q-table
    q_table[state,action] = q_table[state,action]*(1-learning_rate)+(learning_rate*(reward+discount_rate*np.max(q_table[new_state,:])))
    state = new_state
    rewards_current_episode += reward
    if done == True:
      break
    exploration_rate = min_exploration_rate + \
    (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episode)
    rewards_all_episodes.append(rewards_current_episode)

print("************ Training complete ************")

************ Training complete ************


In [6]:
q_table.shape

(16, 4)

In [7]:
#Calculate and print average reward per thousand episodes
rewards_per_thousand_episodes = np.array_split(np.array(rewards_all_episodes), num_episodes/1000)

count = 1000
print("Average per thousand episodes")
for r in rewards_per_thousand_episodes: 
  print(count," : ",str(sum(r/1000)))
  count += 1000

Average per thousand episodes
1000  :  0.0
2000  :  0.0
3000  :  0.0
4000  :  0.0
5000  :  0.0
6000  :  0.0
7000  :  0.0
8000  :  0.0
9000  :  0.0
10000  :  0.0


In [8]:
#Visualize our Agent
import time
for episode in range(3):
  state = env.reset()
  done = False
  print("Episode is: " + str(episode))
  time.sleep(3)

  for step in range(max_steps_per_episode):
    clear_output(wait=True)
    env.render()
    action = np.argmax(q_table[state,:])
    time.sleep(.21)

    new_state, reward, done, info = env.step(action)
    if done:
      clear_output(wait=True)
      env.render()
      if reward == 1:
        print("**** Reached Goal")
        time.sleep(2)
      else:
        print("**** Failed")
        time.sleep(2)
      break

    state = new_state

env.close()



  (Down)
SFFF
FHFH
FFFH
HFFG
**** Reached Goal
